In [1]:
%load_ext autoreload
%autoreload 2

import faiss
import pickle
import numpy as np
import os
from tools.utils import draw_bbboxes
from pycocotools.coco import COCO


In [2]:
print(os.getcwd())
OUTPUT_PATH="images/threshold_study/final_feature_db_on_train.npy"
feautre_db = np.load(OUTPUT_PATH)
coco = COCO("/home.nfs/babayeln/thesis/mask-rcnn.pytorch/data/coco/annotations/instances_train2017.json")

/home.nfs/babayeln/doc
loading annotations into memory...
Done (t=17.11s)
creating index...
index created!


In [3]:
coco_cat_to_continous_cat = {v: i+1 for i,v in enumerate(coco.cats)}
continious_cat_to_coco = {v:k for k,v in coco_cat_to_continous_cat.items()}

In [13]:
def find_threhold_for_each_class(index, db, classes,  k_neighbours=10):
    print("Doing search")
    distance, indecies = index.search(db, k_neighbours)
    print("Finishing search")
    classes_idx = classes[indecies]
    distance_class = {}
    counts = {}
    for idx, neighbours in enumerate(classes_idx):
        myself = int(classes[idx])
        not_class_neighbours = np.where(neighbours!=myself)[0]
        if len(not_class_neighbours) == 0:
            not_class_neighbours = [k_neighbours-1]
        first_not_class_neighbours = not_class_neighbours[0]
        #if first_not_class_neighbours==0:
        #    import ipdb; ipdb.set_trace()
        if myself not in counts.keys():
            counts[myself] = []
            distance_class[myself] = []
        counts[myself].append(first_not_class_neighbours)
        distance_class[myself].append(distance[idx, first_not_class_neighbours])
        
    average_distance_class = {}
    for class_idx in distance_class.keys():
        average_distance_class[class_idx] = np.median(distance_class[class_idx])
    return average_distance_class, counts, classes_idx, distance, indecies


def xyxy_to_xywh(xyxy):
    """Convert [x1 y1 x2 y2] box format to [x1 y1 w h] format."""
    if isinstance(xyxy, (list, tuple)):
        # Single box given as a list of coordinates
        assert len(xyxy) == 4
        x1, y1 = xyxy[0], xyxy[1]
        w = xyxy[2] - x1 + 1
        h = xyxy[3] - y1 + 1
        return (x1, y1, w, h)
    elif isinstance(xyxy, np.ndarray):
        # Multiple boxes given as a 2D ndarray
        return np.hstack((xyxy[:, 0:2], xyxy[:, 2:4] - xyxy[:, 0:2] + 1))
    else:
        raise TypeError('Argument xyxy must be a list, tuple, or numpy array.')


In [14]:
def create_db(db):
    dimension = 1024
    db = db.astype('float32')
    faiss_db = faiss.IndexFlatL2(dimension)
    faiss_db.add(db)
    return faiss_db
"""
def create_db(database, M=128, nbits=8, nlist=316, nprobe=32):
    quantizer = faiss.IndexFlatL2(1024)
    #index = faiss.IndexIVFPQ(quantizer, 1024, nlist, M, nbits)
    index = faiss.IndexIVFFlat(quantizer, 1024, nlist)
    N = int(len(database))
    samples = database[np.random.permutation(np.arange(N))[:N]]
    index.train(samples)
    index.add(database)
    return index"""

'\ndef create_db(database, M=128, nbits=8, nlist=316, nprobe=32):\n    quantizer = faiss.IndexFlatL2(1024)\n    #index = faiss.IndexIVFPQ(quantizer, 1024, nlist, M, nbits)\n    index = faiss.IndexIVFFlat(quantizer, 1024, nlist)\n    N = int(len(database))\n    samples = database[np.random.permutation(np.arange(N))[:N]]\n    index.train(samples)\n    index.add(database)\n    return index'

In [15]:
faiss_db = create_db(np.array(feautre_db[:, 7:]))


In [16]:
classes = np.array(feautre_db[:, 2])
features = np.array(feautre_db[:, 7:])

In [ ]:
#classes_idx, indecies,  distance = find_threhold_for_each_class(faiss_db, features, classes,  k_neighbours=10)

%time average_distance_class, counts, classes_idx, distances, indecies = find_threhold_for_each_class(faiss_db, features, classes,  k_neighbours=5)

Doing search


In [ ]:
with open("images/threshold_study/nearest_neighbour_5.pkl", "wb") as f:
    pickle.dump({
        "average_distance_class" : average_distance_class,
        "counts" : counts,
        "classes_idx": classes_idx,
        "distances": distances,
        "indecies" : indecies
        }, f)

In [19]:
indecies

array([[     0, 819276, 135478, 652556, 360902],
       [     1, 266078,      3, 684495, 485016],
       [     2,  31170, 147105,  59081, 134175],
       ...,
       [859950, 119011, 387249, 209479, 713814],
       [859951,  61269, 390625, 512074, 454202],
       [859952, 541633,  81518, 540341, 118740]])

## Chossing different classes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def get_top_x_not_weighted(x):
    matrix = np.zeros(shape = (81, 81))
    weights = [1, 1, 1, 1, 1]
    for idx, neighbours in enumerate(classes_idx):
        neighbours = neighbours[:x+1]
        for position, neighbour in enumerate(neighbours[1:]):
            #if neighbour!=classes[idx]:
            matrix[int(classes[idx]), int(neighbour)] += weights[position] 
    for idx in range(matrix.shape[0]):
        matrix[idx] = np.round(matrix[idx]/ np.sum(matrix[idx], axis=0),4)
    return matrix

def get_df(matrix):
    import pandas as pd
    df = pd.DataFrame(matrix)
    names = ["background"] + [coco.cats[coco_cat]["name"] for (cat, coco_cat) in continious_cat_to_coco.items()]
    df.set_axis(names, axis=0)
    df.set_axis(names, axis=1)
    df.style.background_gradient()
    return df

def draw_df(df, name, figsize=(20, 20), ann=False ):
    fig, ax = plt.subplots(figsize=figsize)
    pal = sns.cubehelix_palette(light=1, as_cmap=True)

    sns_plot = sns.heatmap(ax = ax, data=df, cmap=pal, annot=ann)
    sns_plot.get_figure().savefig("nn_review/" + name)


In [ ]:
m_5 = get_top_x_not_weighted(5)
d_5 = get_df(m_5)
draw_df(d, "top5.png")

In [ ]:
m = get_top_x_not_weighted(1)
d = get_df(m)
draw_df(d, "top1.png")

In [ ]:
m_5[1], m[1]

In [ ]:
df.to_pickle("nn_review/nn_to_wrong_classes_5.pkl")
df.style.background_gradient().to_excel("nn_review/nn_to_wrong_classes_5.xlsx")

## Chosing yourself

In [ ]:
matrix = np.zeros(shape = (81,4))
for idx, neighbours in enumerate(classes_idx):
    for position, neighbour in enumerate(neighbours[1:]):
        if neighbour==classes[idx]:
            matrix[int(neighbour)][position]+=1
for idx in range(matrix.shape[0]):
    matrix[idx] = np.round(matrix[idx]/ np.sum(matrix[idx]),3)*100

In [ ]:
np.sum(matrix[5])

In [ ]:
import pandas as pd
df = pd.DataFrame(matrix)
names = ["background"] + [coco.cats[coco_cat]["name"] for (cat, coco_cat) in continious_cat_to_coco.items()]
df.set_axis(names, axis=0)
df.style.background_gradient()
draw_df(df, "same_class_%", figsize=(5,20), ann=True)

In [ ]:
df.to_pickle("nn_review/class_place_apperance.pkl")
df.style.background_gradient().to_excel("nn_review/class_place_apperance.xlsx")

## Mean distance

In [ ]:
names = [coco.cats[continious_cat_to_coco[x]]["name"] for x in average_distance_class.keys()]

In [ ]:
import matplotlib.pyplot as plt
#plt.figure(figsize=(20, 20))

#plt.bar(average_distance_class.keys(), average_distance_class.values())
dictionary = {"class": [], "mean_distance": []}
for key,value in average_distance_class.items():
    dictionary["class"].append(coco.cats[continious_cat_to_coco[key]]["name"])
    dictionary["mean_distance"].append(value)
df = pd.DataFrame(dictionary)
df = df.sort_values('mean_distance')
ax = df.plot.barh(x='class', y='mean_distance', rot=0, figsize=(20,20))
fig = ax.get_figure()

fig.savefig("nn_review/median_distance.png")

#x = list(range(81))
#my_xticks = [coco.cats[coco_cat]["name"] for (cat, coco_cat) in continious_cat_to_coco.items()]
#plt.xticks(x, my_xticks, rotation=45)

#plt.show()

## Distance to different classes

In [ ]:
tmp_dict= {}
for idx in range(len(distances)):
    me = int(classes[idx])
    for distance,other in zip(distances[idx], classes_idx[idx]):
        key = (me, int(other))
        if key not in tmp_dict.keys():
            tmp_dict[key] = []
        if distance<0:
            distance=0
        tmp_dict[key].append(distance)    

In [ ]:
matrix = np.zeros((81, 81))
for (me, other) in tmp_dict.keys():
    matrix[me][other] = np.median(tmp_dict[(me, other)])

In [ ]:
import pandas as pd
df = pd.DataFrame(matrix)
names = ["background"] + [coco.cats[coco_cat]["name"] for (cat, coco_cat) in continious_cat_to_coco.items()]
df.set_axis(names, axis=0)
df.set_axis(names, axis=1)
df.style.background_gradient()
draw_df(df, name="median_distance", ann=False)

In [ ]:
df.to_pickle("nn_review/mean_distance_to_different_classes.pkl")
df.style.background_gradient().to_excel("nn_review/mean_distance_to_different_classes.xlsx")

In [ ]:
def choose_gradient_id_for_background(faiss_db, feautre_db, looked_features, thresholds, k_neighbours ):
    distance, indecies = faiss_db.search(looked_features, k_neighbours)
    #feautre_db[indecies].shape -> number of features, k_neighbours, 1031 dimension vector
    classes = feautre_db[indecies][:, :, 2]
    max_classes = [np.argmax(np.bincount(sample.astype("int64"))) for sample in classes]
    class_thresholds = thresholds[max_classes]
    passed_thresholds = np.where( np.min(distance, axis=1) < class_thresholds)[0]
    background_gradient_idx = np.array(set(range(len(looked_features))) - set(passed_thresholds))
    return background_gradient_idx

In [ ]:
background_gradient_idx = choose_gradient_id_for_background(faiss_db, feautre_db, looked_features, thresholds, k_neighbours )

In [ ]:

background_gradient_idx = choose_gradient_id_for_background(faiss_db, feautre_db, looked_features, thresholds, k_neighbours )